In [1]:
import cv2
import numpy as np
from PIL import Image
import mediapipe as mp

# 웹캡처
cap = cv2.VideoCapture(0)

# 옷 이미지 불러오기 (예: T-shirt)
shirt_image = Image.open("shirt.png").convert("RGBA")  # RGBA 형식으로 변환

# Mediapipe를 사용하여 얼굴 및 어깨 추적
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
face_mesh = mp_face_mesh.FaceMesh()

# 웹캡 실행
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # BGR을 RGB로 변환
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = face_mesh.process(frame_rgb)

    if result.multi_face_landmarks:
        landmarks = result.multi_face_landmarks[0]

        # 어깨 랜드마크 인덱스
        left_shoulder_index = 11  # 좌측 어깨
        right_shoulder_index = 12  # 우측 어깨
        neck_index = 7  # 목 (C7)

        # 좌표 계산
        shoulder_left = (landmarks.landmark[left_shoulder_index].x * frame.shape[1],
                         landmarks.landmark[left_shoulder_index].y * frame.shape[0])
        shoulder_right = (landmarks.landmark[right_shoulder_index].x * frame.shape[1],
                          landmarks.landmark[right_shoulder_index].y * frame.shape[0])
        neck = (landmarks.landmark[neck_index].x * frame.shape[1],
                landmarks.landmark[neck_index].y * frame.shape[0])

        # 어깨 너비 계산
        shoulder_width = np.linalg.norm(np.array(shoulder_left) - np.array(shoulder_right))

        # 어깨 너비가 너무 작을 경우 최소값 설정 (예: 50px)
        if shoulder_width < 50:
            shoulder_width = 50

        # 상체 높이를 계산 (목에서 어깨까지의 거리)
        upper_body_height = np.linalg.norm(np.array(shoulder_left) - np.array(neck))

        # 옷 이미지 크기 동적 조정
        shirt_width = int(shoulder_width)  # 어깨 너비에 맞춰서 옷 크기 결정
        shirt_height = int(upper_body_height * 1.5)  # 상체 높이에 비례하여 옷 길이 설정

        # 옷 이미지 크기 조정
        shirt_resized = np.array(shirt_image.resize((shirt_width, shirt_height)))

        # 옷 배치를 위해 목표 위치 계산 (목과 어깨 사이)
        shirt_top = neck[1]  # 목의 위치
        shirt_bottom = shirt_top + shirt_height  # 옷의 하단 위치

        # 아핀 변환을 위한 점 정의
        src_points = np.array([[0, 0], [shirt_width, 0], [0, shirt_height]], dtype='float32')
        dst_points = np.array([[shoulder_left[0], shirt_top], 
                                [shoulder_right[0], shirt_top], 
                                [shoulder_left[0], shirt_bottom]], 
                               dtype='float32')

        # 아핀 변환 행렬 계산
        M = cv2.getAffineTransform(src_points, dst_points)

        # 옷 이미지 변환
        transformed_shirt = cv2.warpAffine(shirt_resized[:, :, :3], M, (frame.shape[1], frame.shape[0]))

        # 투명도 체크하여 프레임에 합성
        for i in range(shirt_height):
            for j in range(shirt_width):
                # 변환된 좌표 계산
                new_x = int(dst_points[2][0]) + j
                new_y = int(dst_points[2][1]) + i

                if 0 <= new_x < frame.shape[1] and 0 <= new_y < frame.shape[0]:  # 범위 확인
                    alpha = shirt_resized[i, j][3] / 255.0  # 알파 채널 값 (0~255)을 0~1로 변환
                    if alpha > 0:  # 투명도 체크
                        frame[new_y, new_x] = (
                            frame[new_y, new_x] * (1 - alpha) +
                            transformed_shirt[new_y, new_x] * alpha
                        )  # 배경과 옷을 혼합

    # 결과 출력
    cv2.imshow("Webcam with Outfit", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 